In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *

sc = SparkContext.getOrCreate()
ss = SparkSession.builder.getOrCreate()

In [ ]:
status = sc.textFile("../Data/bike_share/status_million.csv")

## schema : station_id (int), num_bikes_available (int), num_docks_available (int), timestamp(timestamp)

In [ ]:
from datetime import datetime
def toIntSafe(inval):
    try:
        return int(inval)
    except:
        return None

def toTimeSafe(inval):
    inval = inval.strip("\"") # Timestamp starting and ending with a double quotation mark.
    try:
        return datetime.strptime(inval, "%Y-%m-%d %H:%M:%S")
    except:
        return None

In [ ]:
schema = StructType([ StructField("station_id", IntegerType(), False),
                      StructField("num_bikes_available", IntegerType(), True),
                      StructField("num_docks_available", IntegerType(), True),
                      StructField("timestamp", TimestampType(), True)
                    ])

In [ ]:
status_transformed = status.map(lambda x : x.split(","))\
                           .map(lambda x : (int(x[0]), toIntSafe(x[1]), toIntSafe(x[2]), toTimeSafe(x[3])))

In [ ]:
status_df = ss.createDataFrame(status_transformed, schema)

In [ ]:
status_df.show(5)

In [ ]:
status_df.filter('station_id == 10').show()

## Return the current and previous number of bike at station_id, 10 at each time data was collected order by timestamp.

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import *